In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# !git clone https://github.com/emiliodelgadouy/utils.git

In [ ]:
from matplotlib import pyplot as plt
from utils.PatchSequence import PatchSequence
from utils.Dataset import Dataset

In [ ]:
dataset = Dataset(lateralize=False, reduced=True, n=500)

In [ ]:
# from google.colab import auth
# auth.authenticate_user()
# !sudo pip install --no-cache-dir -U crcmod
# !gsutil cp gs://helen-data/dumps/dump_cache_reduced_half.npz ./

In [ ]:
dataset.load_dump('dump_cache_reduced.npz')

In [ ]:
ds_training = dataset[dataset["split"] == "training"]
ds_test = dataset[dataset["split"] == "test"]

In [ ]:
!pip install comet_ml optuna
from comet_ml import Experiment
import optuna
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.utils import class_weight


def objective(trial):
    # Crear experimento en Comet
    experiment = Experiment(
        api_key="W8IluWhLfvXgLr8ggg1bwZKTf",
        project_name="prueba",
        workspace="emiliodelgadouy",
        auto_output_logging="simple",
        log_code=False
    )
    experiment.set_name(f"trial_{trial.number}")

    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.6)
    l2_reg = trial.suggest_float('l2_reg', 1e-6, 1e-3, log=True)
    unfreeze_pct = trial.suggest_float("unfreeze_pct", 0.0, 0.2)

    IMG_SIZE = (299, 299)
    BATCH_SIZE = 32
    EPOCHS = 1

    experiment.log_parameters({
        "learning_rate": learning_rate,
        "dropout_rate": dropout_rate,
        "l2_reg": l2_reg,
        "backbone": "InceptionV3",
        "batch_size": BATCH_SIZE,
        "img_size": IMG_SIZE,
        "epochs": EPOCHS
    })

    # Generadores
    train_generator = PatchSequence(
        ds_training,
        batch_size=BATCH_SIZE,
        patch_size=IMG_SIZE,
        shuffle=True,
        preprocess_fn=preprocess_input
    )
    val_generator = PatchSequence(
        ds_test,
        batch_size=BATCH_SIZE,
        patch_size=IMG_SIZE,
        shuffle=False,
        preprocess_fn=preprocess_input
    )

    # Backbone
    base = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )
    for layer in base.layers:
        layer.trainable = False
    n_total = len(base.layers)
    n_unfreeze = int(n_total * unfreeze_pct)

    if n_unfreeze > 0:
        for layer in base.layers[-n_unfreeze:]:
            layer.trainable = True

    labels = np.array(ds_training["findings"])

    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(labels),
        y=labels
    )
    class_weights = {i: w for i, w in enumerate(weights)}
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(
        512,
        activation='relu',
        kernel_regularizer=l1_l2(l1=1e-4, l2=l2_reg)
    )(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc_roc', curve='ROC')]
    )

    t0 = time.time()
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        steps_per_epoch=len(train_generator),
        validation_steps=len(val_generator),
        class_weight=class_weights,
        verbose=0
    )
    elapsed = time.time() - t0

    experiment.log_metric("training_time_sec", elapsed)
    experiment.log_parameter("unfreeze_pct", unfreeze_pct)


    for i in range(EPOCHS):
        experiment.log_metric("train_accuracy", history.history["accuracy"][i], step=i)
        experiment.log_metric("train_auc_roc", history.history["auc_roc"][i], step=i)
        experiment.log_metric("train_loss", history.history["loss"][i], step=i)

        experiment.log_metric("val_accuracy", history.history["val_accuracy"][i], step=i)
        experiment.log_metric("val_auc_roc", history.history["val_auc_roc"][i], step=i)
        experiment.log_metric("val_loss", history.history["val_loss"][i], step=i)


    # Evaluación final
    score = model.evaluate(val_generator, steps=len(val_generator), verbose=0)
    experiment.log_metric("final_val_loss", score[0])
    experiment.log_metric("final_val_accuracy", score[1])
    experiment.log_metric("final_val_auc_roc", score[2])


    summary = f"""
    Trial #{trial.number}
    learning_rate: {learning_rate}
    dropout_rate: {dropout_rate}
    l2_reg: {l2_reg}
    unfreeze_pct: {unfreeze_pct}
    """
    experiment.log_other("trial_summary", summary)


    experiment.end()
    return score[2]  # AUC ROC


In [ ]:
study = optuna.create_study(direction='maximize', study_name='optuna_inception')
study.optimize(objective, n_trials=1)

print("🔍 Mejor trial:")
print("  AUC ROC:", study.best_value)
print("  Hiperparámetros:", study.best_params)

[W 2025-08-06 09:11:57,987] Trial 0 failed with parameters: {'learning_rate': 0.0005110200433190774, 'dropout_rate': 0.4998881189816316, 'l2_reg': 1.5492791129371202e-06, 'unfreeze_pct': 0.04412156938814387} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/projects/tesis/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_12431/3633953569.py", line 103, in objective
    history = model.fit(
              ^^^^^^^^^^
  File "/opt/projects/tesis/.venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/projects/tesis/.venv/lib/python3.11/site-packages/keras/src/backend/tensorflow/trainer.py", line 377, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/o

KeyboardInterrupt: 